<a href="https://colab.research.google.com/github/cnitin041/Text-to-Image/blob/main/Ninja_Imagetoavatar_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition Pillow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=63e776b28fa99d2e9756ed81865d9e2296464372ecf8bc5520c838fdc1d27b5c
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
import random
import io
import base64
from google.colab import files
from IPython.display import display, HTML

def create_avatar(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use a face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        print("No face detected in the image.")
        return None

    # Get the first face
    (x, y, w, h) = faces[0]

    # Create a PIL Image to draw on
    pil_image = Image.new('RGB', (200, 200), color='white')
    d = ImageDraw.Draw(pil_image)

    # Draw face shape
    face_color = (random.randint(200, 255), random.randint(150, 200), random.randint(100, 150))
    d.ellipse([(20, 20), (180, 180)], fill=face_color, outline='black')

    # Draw eyes
    eye_color = (random.randint(0, 100), random.randint(0, 100), random.randint(0, 100))
    d.ellipse([(60, 80), (90, 100)], fill='white', outline='black')
    d.ellipse([(110, 80), (140, 100)], fill='white', outline='black')
    d.ellipse([(70, 85), (80, 95)], fill=eye_color)
    d.ellipse([(120, 85), (130, 95)], fill=eye_color)

    # Draw nose
    d.line([(100, 110), (100, 130)], fill='black', width=2)

    # Draw mouth
    d.arc((70, 130, 130, 170), 0, 180, fill='red', width=2)

    return pil_image

def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Function to handle file upload and avatar creation
def create_avatars_from_uploads():
    uploaded = files.upload()
    avatars = []

    for filename in uploaded.keys():
        avatar = create_avatar(filename)
        if avatar:
            avatars.append((filename, avatar))

    # Display avatars
    for filename, avatar in avatars:
        print(f"Avatar for {filename}:")
        display(avatar)

    # Create a download link for each avatar
    for filename, avatar in avatars:
        img_base64 = image_to_base64(avatar)
        download_link = f'<a href="data:image/png;base64,{img_base64}" download="{filename}_avatar.png">Download {filename} avatar</a>'
        display(HTML(download_link))

# Call the function to start the process
create_avatars_from_uploads()

KeyboardInterrupt: 

In [3]:
!pip install dlib opencv-python-headless pillow numpy


In [4]:
import cv2
import dlib
import numpy as np
from PIL import Image, ImageDraw
import random
import io
import base64
from google.colab import files
from IPython.display import display, HTML

# Download the shape predictor model
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

def get_face_landmarks(image_path):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    if len(faces) == 0:
        return None

    landmarks = predictor(gray, faces[0])
    return [(p.x, p.y) for p in landmarks.parts()]

def create_avatar(image_path):
    landmarks = get_face_landmarks(image_path)
    if landmarks is None:
        print("No face detected in the image.")
        return None

    pil_image = Image.new('RGB', (400, 400), color='white')
    d = ImageDraw.Draw(pil_image)

    # Face shape
    jaw = landmarks[0:17]
    d.polygon(jaw, fill='peachpuff', outline='black')

    # Eyes
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    d.polygon(left_eye, fill='white', outline='black')
    d.polygon(right_eye, fill='white', outline='black')

    # Eyebrows
    left_eyebrow = landmarks[17:22]
    right_eyebrow = landmarks[22:27]
    d.line(left_eyebrow, fill='black', width=2)
    d.line(right_eyebrow, fill='black', width=2)

    # Nose
    nose = landmarks[27:36]
    d.line(nose, fill='black', width=2)

    # Mouth
    outer_lip = landmarks[48:60]
    inner_lip = landmarks[60:68]
    d.polygon(outer_lip, fill='pink', outline='black')
    d.polygon(inner_lip, fill='red', outline='black')

    # Glasses (just for fun)
    left_eye_center = np.mean(left_eye, axis=0).astype(int)
    right_eye_center = np.mean(right_eye, axis=0).astype(int)
    glasses_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    d.line([tuple(left_eye_center), tuple(right_eye_center)], fill=glasses_color, width=3)
    d.ellipse([tuple(left_eye_center - np.array([20, 15])), tuple(left_eye_center + np.array([20, 15]))], outline=glasses_color, width=3)
    d.ellipse([tuple(right_eye_center - np.array([20, 15])), tuple(right_eye_center + np.array([20, 15]))], outline=glasses_color, width=3)

    return pil_image

def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

def create_avatars_from_uploads():
    uploaded = files.upload()
    avatars = []

    for filename in uploaded.keys():
        avatar = create_avatar(filename)
        if avatar:
            avatars.append((filename, avatar))

    for filename, avatar in avatars:
        print(f"Avatar for {filename}:")
        display(avatar)

    for filename, avatar in avatars:
        img_base64 = image_to_base64(avatar)
        download_link = f'<a href="data:image/png;base64,{img_base64}" download="{filename}_avatar.png">Download {filename} avatar</a>'
        display(HTML(download_link))

create_avatars_from_uploads()

--2024-07-18 07:35:30--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  39.1MB/s    in 1.6s    

2024-07-18 07:35:32 (39.1 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



KeyboardInterrupt: 

In [5]:
!pip install diffusers==0.10.2 transformers ftfy scipy accelerate
!pip install -q https://github.com/huggingface/diffusers/archive/main.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.1/503.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 31.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Image

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# If not using GPU, print a message suggesting to switch
if device == "cpu":
    print("WARNING: You're not using a GPU. The process will be very slow.")
    print("Go to Runtime > Change runtime type and select GPU for better performance.")

# Model ID
model_id = "CompVis/stable-diffusion-v1-4"

# Load the pipeline
if device == "cuda":
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
else:
    pipe = StableDiffusionPipeline.from_pretrained(model_id)

pipe = pipe.to(device)

# Generate an image
prompt = "A powerful and haunting portrait of Mother Nature, depicted as an elderly woman with a face covered in soot and grime. She holds a dying tree in her arms, her eyes filled with sorrow. The background shows a desolate and polluted landscape, littered with trash and debris. Humans walk by, oblivious to her suffering, immersed in their phones or conversations. The color palette is muted and somber, reflecting the bleakness and despair of the scene."
image = pipe(prompt).images[0]

# Display the image
display(image)

# Save the image
image.save("your.png")
print("your.png'")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cpu
Go to Runtime > Change runtime type and select GPU for better performance.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (96 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['color palette is muted and somber, reflecting the bleakness and despair of the scene.']


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Image
import os
from PIL import Image as PILImage

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# If not using GPU, print a message suggesting to switch
if device == "cpu":
    print("WARNING: You're not using a GPU. The process will be very slow.")
    print("Go to Runtime > Change runtime type and select GPU for better performance.")

# Model ID for 3D avatar generation
model_id = "your_3d_avatar_model_id"  # Replace with the actual model ID for 3D avatar generation

# Load the pipeline
if device == "cuda":
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
else:
    pipe = StableDiffusionPipeline.from_pretrained(model_id)

pipe = pipe.to(device)

# Function to get user input for photos
def get_user_photos():
    photos = []
    for i in range(5):
        photo_path = input(f"Enter the path for photo {i+1}: ")
        if os.path.exists(photo_path):
            photos.append(PILImage.open(photo_path))
        else:
            print(f"File not found: {photo_path}")
    return photos

# Get user photos
user_photos = get_user_photos()

# Generate 3D avatar
avatar = pipe(user_photos).images[0]

# Display the avatar
display(avatar)

# Save the avatar
avatar.save("3d_avatar.png")
print("3D avatar saved as '3d_avatar.png'")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cpu
Go to Runtime > Change runtime type and select GPU for better performance.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-66911c38-30dd38167caceaad3cffa473;4b437aa7-fd15-4a24-9d2a-fc3f545bc427)

Repository Not Found for url: https://huggingface.co/api/models/your_3d_avatar_model_id.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password..
Will try to load from local cache.


OSError: Cannot load model your_3d_avatar_model_id: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.

In [ ]:
prompt = """
A close-up portrait of a young woman with freckles and long, flowing hair. She is standing against a backdrop of a rustic wooden fence. The lighting is soft and natural, highlighting her features and creating a gentle shadow on one side of her face. She has a slight smile and her eyes are looking directly into the camera
"""

image = pipe(prompt, height=512, width=512).images[0]
display(image)

# Optionally, save the image
image.save("avatar.png")

NameError: name 'pipe' is not defined